In [1]:
from aocd import get_data

puzzle_input = get_data(day=5, year=2023)

In [2]:
it = """
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
""".strip()

Partie 1

Partie 2

In [3]:
import re
from typing import Optional
from dataclasses import dataclass

@dataclass
class Segment:
    start: int
    end: int

    mapping: Optional[int] = None

    @property
    def length(self) -> int:
        return self.end - self.start

    def __lt__(self, seg: "Segment") -> bool:
        return (self.start, self.end) < (seg.start, seg.end)


def parse_input(input: str):
    elements = input.split("\n\n")

    seeds = [int(n) for n in re.findall(r"\d+", elements[0].split(":")[1])]
    seeds_segments = sorted([
        Segment(seed_start, seed_start + length)
        for seed_start, length in zip(seeds[::2], seeds[1::2])
    ])

    maps = []
    for element in elements[1:]:

        mapping_segments = []
        for row in element.split("\n")[1:]:
            dest_start, source_start, length = map(int, re.findall(r"\d+", row))

            segment = Segment(start=source_start, end=source_start+length, mapping=dest_start)
            mapping_segments.append(segment)

        maps.append(sorted(mapping_segments))

    return {
        "seeds": seeds_segments,
        "maps": maps
    }

pit = parse_input(it)
pit

{'seeds': [Segment(start=55, end=68, mapping=None),
  Segment(start=79, end=93, mapping=None)],
 'maps': [[Segment(start=50, end=98, mapping=52),
   Segment(start=98, end=100, mapping=50)],
  [Segment(start=0, end=15, mapping=39),
   Segment(start=15, end=52, mapping=0),
   Segment(start=52, end=54, mapping=37)],
  [Segment(start=0, end=7, mapping=42),
   Segment(start=7, end=11, mapping=57),
   Segment(start=11, end=53, mapping=0),
   Segment(start=53, end=61, mapping=49)],
  [Segment(start=18, end=25, mapping=88),
   Segment(start=25, end=95, mapping=18)],
  [Segment(start=45, end=64, mapping=81),
   Segment(start=64, end=77, mapping=68),
   Segment(start=77, end=100, mapping=45)],
  [Segment(start=0, end=69, mapping=1), Segment(start=69, end=70, mapping=0)],
  [Segment(start=56, end=93, mapping=60),
   Segment(start=93, end=97, mapping=56)]]}

In [4]:
def sources_to_dest_segments(sources_segs: list[Segment], cut_segs: list[Segment]):
    seeds = [*sources_segs]
    new_seeds = []

    while seeds:
        source_seg = seeds.pop()
        intersection = False

        for cut_seg in cut_segs:
            intersect_start = max(source_seg.start, cut_seg.start)
            intersect_end = min(source_seg.end, cut_seg.end)

            # intersection
            if intersect_start < intersect_end:
                intersection = True

                intersect_length = intersect_end - intersect_start

                diff_start = intersect_start - cut_seg.start

                new_seeds.append(
                    Segment(
                        start=cut_seg.mapping + diff_start,
                        end=cut_seg.mapping + intersect_length + diff_start,
                    )
                )

                if source_seg.start < intersect_start:
                    seeds.append(Segment(start=source_seg.start, end=intersect_start))
                if intersect_end < source_seg.end:
                    seeds.append(Segment(start=intersect_end, end=source_seg.end))

                break

        if not intersection:
            new_seeds.append(source_seg)

    return new_seeds


sources_to_dest_segments(
    [Segment(start=55, end=68, mapping=None), Segment(start=79, end=93, mapping=None)],
    [Segment(start=50, end=98, mapping=52), Segment(start=98, end=100, mapping=50)],
)

[Segment(start=81, end=95, mapping=None),
 Segment(start=57, end=70, mapping=None)]

In [5]:
def transform_seeds(seeds: list[Segment], maps: list[list[Segment]]):
    for cut_segs in maps:
        seeds = sources_to_dest_segments(seeds, cut_segs)

    return seeds

min(s.start for s in transform_seeds(pit["seeds"], pit["maps"]))

46

In [6]:
parsed_input = parse_input(puzzle_input)
min(s.start for s in transform_seeds(parsed_input["seeds"], parsed_input["maps"]))

9622622